# 多继承以及 MRO 顺序

## 1. 多继承中调用父类方式不同结果不同

### 1.1 单独调用父类的方法


In [4]:
# coding=utf-8

print("******多继承使用类名.__init__ 发生的状态******")


class Parent(object):
    def __init__(self, name):
        print("parent的init开始被调用")
        self.name = name
        print("parent的init结束被调用")


class Son1(Parent):
    def __init__(self, name, age):
        print("Son1的init开始被调用")
        self.age = age
        Parent.__init__(self, name)
        print("Son1的init结束被调用")


class Son2(Parent):
    def __init__(self, name, gender):
        print("Son2的init开始被调用")
        self.gender = gender
        Parent.__init__(self, name)
        print("Son2的init结束被调用")


class Grandson(Son1, Son2):
    def __init__(self, name, age, gender):
        print("Grandson的init开始被调用")
        Son1.__init__(self, name, age)  # 单独调用父类的初始化方法
        Son2.__init__(self, name, gender)
        print("Grandson的init结束被调用")


gs = Grandson("grandson", 12, "男")
print("姓名：", gs.name)
print("年龄：", gs.age)
print("性别：", gs.gender)

print("******多继承使用类名.__init__ 发生的状态******\n\n")

******多继承使用类名.__init__ 发生的状态******
Grandson的init开始被调用
Son1的init开始被调用
parent的init开始被调用
parent的init结束被调用
Son1的init结束被调用
Son2的init开始被调用
parent的init开始被调用
parent的init结束被调用
Son2的init结束被调用
Grandson的init结束被调用
姓名： grandson
年龄： 12
性别： 男
******多继承使用类名.__init__ 发生的状态******




### 1.2 多继承中 super 调用被重写的父类方法


In [5]:
print("******多继承使用super().__init__ 发生的状态******")


class Parent(object):
    def __init__(self, name, *args, **kwargs):  # 为避免多继承报错，使用不定长参数，接受参数
        print("parent的init开始被调用")
        self.name = name
        print("parent的init结束被调用")


class Son1(Parent):
    def __init__(self, name, age, *args, **kwargs):  # 为避免多继承报错，使用不定长参数，接受参数
        print("Son1的init开始被调用")
        self.age = age
        super().__init__(name, *args, **kwargs)  # 为避免多继承报错，使用不定长参数，接受参数
        print("Son1的init结束被调用")


class Son2(Parent):
    def __init__(self, name, gender, *args, **kwargs):  # 为避免多继承报错，使用不定长参数，接受参数
        print("Son2的init开始被调用")
        self.gender = gender
        super().__init__(name, *args, **kwargs)  # 为避免多继承报错，使用不定长参数，接受参数
        print("Son2的init结束被调用")


class Grandson(Son1, Son2):
    def __init__(self, name, age, gender):
        print("Grandson的init开始被调用")
        # 多继承时，相对于使用类名.__init__方法，要把每个父类全部写一遍
        # 而super只用一句话，执行了全部父类的方法，这也是为何多继承需要全部传参的一个原因
        # super(Grandson, self).__init__(name, age, gender)
        super().__init__(name, age, gender)
        print("Grandson的init结束被调用")


print(Grandson.__mro__)

gs = Grandson("grandson", 12, "男")
print("姓名：", gs.name)
print("年龄：", gs.age)
print("性别：", gs.gender)
print("******多继承使用super().__init__ 发生的状态******\n\n")

******多继承使用super().__init__ 发生的状态******
(<class '__main__.Grandson'>, <class '__main__.Son1'>, <class '__main__.Son2'>, <class '__main__.Parent'>, <class 'object'>)
Grandson的init开始被调用
Son1的init开始被调用
Son2的init开始被调用
parent的init开始被调用
parent的init结束被调用
Son2的init结束被调用
Son1的init结束被调用
Grandson的init结束被调用
姓名： grandson
年龄： 12
性别： 男
******多继承使用super().__init__ 发生的状态******




### 1.3 注意

以上 2 个代码执行的结果不同

如果 2 个子类中都继承了父类，当在子类中通过父类名调用时，parent 被执行了 2 次
如果 2 个子类中都继承了父类，当在子类中通过 super 调用时，parent 被执行了 1 次

## 2.单继承中 super


In [6]:
print("******单继承使用super().__init__ 发生的状态******")


class Parent(object):
    def __init__(self, name):
        print("parent的init开始被调用")
        self.name = name
        print("parent的init结束被调用")


class Son1(Parent):
    def __init__(self, name, age):
        print("Son1的init开始被调用")
        self.age = age
        super().__init__(name)  # 单继承不能提供全部参数
        print("Son1的init结束被调用")


class Grandson(Son1):
    def __init__(self, name, age, gender):
        print("Grandson的init开始被调用")
        super().__init__(name, age)  # 单继承不能提供全部参数
        print("Grandson的init结束被调用")


gs = Grandson("grandson", 12, "男")
print("姓名：", gs.name)
print("年龄：", gs.age)
# print('性别：', gs.gender)
print("******单继承使用super().__init__ 发生的状态******\n\n")

******单继承使用super().__init__ 发生的状态******
Grandson的init开始被调用
Son1的init开始被调用
parent的init开始被调用
parent的init结束被调用
Son1的init结束被调用
Grandson的init结束被调用
姓名： grandson
年龄： 12
******单继承使用super().__init__ 发生的状态******




# 总结

super().**init**相对于类名.**init**，在单继承上用法基本无差

1. 但在多继承上有区别，super 方法能保证每个父类的方法只会执行一次，而使用类名的方法会导致方法被执行多次，具体看前面的输出结果
2. 多继承时，使用 super 方法，对父类的传参数，由于 super 的算法导致的原因，必须把参数全部传递，否则会报错
3. 单继承时，使用 super 方法，则不能全部传递，只能传父类方法所需的参数，否则会报错
4. 多继承时，相对于使用类名.**init**方法，要把每个父类全部写一遍, 而使用 super 方法，只需写一句话便执行了全部父类的方法，这也是为何多继承需要全部传参的一个原因

# 小试牛刀

以下的代码的输出将是什么? 说出你的答案并解释。


In [7]:
class Parent(object):
    x = 1


class Child1(Parent):
    pass


class Child2(Parent):
    pass


print(Parent.x, Child1.x, Child2.x)
Child1.x = 2
print(Parent.x, Child1.x, Child2.x)
Parent.x = 3
print(Parent.x, Child1.x, Child2.x)

1 1 1
1 2 1
3 2 3


使你困惑或是惊奇的是关于最后一行的输出是 3 2 3 而不是 3 2 1。为什么改变了 Parent.x 的值还会改变 Child2.x 的值，但是同时 Child1.x 值却没有改变？

这个答案的关键是，在 Python 中，类变量在内部是作为字典处理的。如果一个变量的名字没有在当前类的字典中发现，将搜索祖先类（比如父类）直到被引用的变量名被找到（如果这个被引用的变量名既没有在自己所在的类又没有在祖先类中找到，会引发一个 AttributeError 异常 ）。

因此，在父类中设置 x = 1 会使得类变量 x 在引用该类和其任何子类中的值为 1。这就是因为第一个 print 语句的输出是 1 1 1。

随后，如果任何它的子类重写了该值（例如，我们执行语句 Child1.x = 2），然后，该值仅仅在子类中被改变。这就是为什么第二个 print 语句的输出是 1 2 1。

最后，如果该值在父类中被改变（例如，我们执行语句 Parent.x = 3），这个改变会影响到任何未重写该值的子类当中的值（在这个示例中被影响的子类是 Child2）。这就是为什么第三个 print 输出是 3 2 3。
